In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
import numpy as np
import datetime, os, wget, requests, unidecode, re
import uuid
import xlwings as xw
from lat_lon_parser import parse
from bs4 import BeautifulSoup

In [66]:
start_date = dt(2020, 11, 8)
end_date = dt(2021, 1, 26)
hours = ['14', '17', '21', '04', '08', '11']
last_day_hours = ['04', '08', '11', '14']

In [10]:
def download_excel(date, hour):
    str_date = date.strftime("%Y%m%d")
    url = f'https://www.vendeeglobe.org/download-race-data/vendeeglobe_{str_date}_{hour}0000.xlsx'
    try:
        if not os.path.exists(f'vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx'):
            wget.download(url, f'vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx')
    except Exception as e:
        print(f'Error for {date.strftime("%Y-%m-%d")} at {hour}H:')
        print(e)

In [11]:
date = start_date
while date <= end_date:
    str_date = date.strftime("%Y%m%d")
    for hour in hours:
        download_excel(date, hour)
    date += datetime.timedelta(days=1)
for hour in last_day_hours:
    download_excel(date, hour)

Error for 2020-11-08 at 04H:
HTTP Error 404: Not Found
Error for 2020-11-08 at 08H:
HTTP Error 404: Not Found
Error for 2020-11-08 at 11H:
HTTP Error 404: Not Found


Faut il lire le premier ? Tous les compétiteurs sont au meme point de départ

In [12]:
date + datetime.timedelta(hours=15)

datetime.datetime(2021, 1, 27, 15, 0)

In [13]:
def clean_excel(date, hour):
    str_date = date.strftime("%Y%m%d")
    data = None
    if os.path.exists(f'vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx'):
        with xw.Book(f"vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx", mode="r") as book:
            sheet1 = book.sheets[0]
            data = sheet1.cells.options("df").value

        data = data.iloc[2:]
        data.iloc[1,:3] = data.iloc[0,:3]
        data.iloc[1,-2:] = data.iloc[0,-2:]
        data.iloc[1] = data.iloc[1].str.split("\r\n").str[0]
        data.iloc[1, 1] = data.iloc[1, 1].replace(" / ",' ')
        data.iloc[1, 2] = data.iloc[1, 2].replace(" / ",'\r\n')
        data.iloc[1,6:10] = data.iloc[1,6:10]+"_30m"
        data.iloc[1,10:14] = data.iloc[1,10:14]+"_last"
        data.iloc[1,14:18] = data.iloc[1,14:18]+"_24h"
        data.columns = data.iloc[1]
        data = data.iloc[2:]
        data = data.iloc[:-4]
        nat_voile = "(?P<Nationalite>.*) (?P<Voile>[0-9]*)"
        data[["Nationalité", "Voile"]] = data['Nat. Voile'].str.extract(nat_voile, expand=True)
        skipper_bateau = "(?P<Skipper>.*)\r\n(?P<nBateau>.*)"
        data[["Skipper", "Bateau"]] = data['Skipper\r\nBateau'].str.extract(skipper_bateau, expand=True)
        data = data.drop(['Nat. Voile', 'Skipper\r\nBateau'], axis=1)
        data["Skipper"] = data["Skipper"].map(lambda x: unidecode.unidecode(x.lower()))
        data["date"] = date + datetime.timedelta(hours=int(hour)) 
        data = data.reset_index(drop=True)
    return data

In [14]:
date = start_date
race_df = pd.DataFrame()
while date <= end_date + datetime.timedelta(days=1):
    for hour in hours:
        tmp_df = clean_excel(date, hour)
        try :
            race_df = pd.concat([race_df, tmp_df])
        except Exception as e:
            print(e)
            break
    date += datetime.timedelta(days=1)
race_df = race_df.dropna(subset=race_df.columns[1:18]).reset_index(drop=True)
race_df['Latitude'] = race_df['Latitude'].apply(parse)
race_df['Longitude'] = race_df['Longitude'].apply(parse)
race_df.head()

,Rang,Heure FR,Latitude,Longitude,Cap_30m,Vitesse_30m,VMG_30m,Distance_30m,Cap_last,Vitesse_last,...,Vitesse_24h,VMG_24h,Distance_24h,DTF,DTL,Nationalité,Voile,Skipper,Bateau,date
0,1,15:30 FR\r\n,46.407667,-1.841333,241°,17.7 kts,17.5 kts,0.3 nm,357°,0.0 kts,...,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm,FRA,18,louis burton,Bureau Vallée 2,2020-11-08 14:00:00
1,2,15:31 FR\r\n1min,46.405667,-1.830333,241°,11.1 kts,10.9 kts,0.4 nm,357°,0.0 kts,...,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm,MON,10,boris herrmann,Seaexplorer - Yacht Club De Monaco,2020-11-08 14:00:00
2,3,15:30 FR\r\n,46.415167,-1.833167,244°,15.5 kts,15.5 kts,0.5 nm,357°,0.0 kts,...,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm,FRA,8,jeremie beyou,Charal,2020-11-08 14:00:00
3,4,15:30 FR\r\n,46.411833,-1.828000,244°,13.2 kts,13.1 kts,0.7 nm,357°,0.0 kts,...,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm,FRA,59,thomas ruyant,LinkedOut,2020-11-08 14:00:00
4,5,15:30 FR\r\n,46.409833,-1.826000,246°,10.9 kts,10.9 kts,0.2 nm,357°,0.0 kts,...,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm,FRA,53,maxime sorel,V And B Mayenne,2020-11-08 14:00:00


In [15]:
race_df

,Rang,Heure FR,Latitude,Longitude,Cap_30m,Vitesse_30m,VMG_30m,Distance_30m,Cap_last,Vitesse_last,...,Vitesse_24h,VMG_24h,Distance_24h,DTF,DTL,Nationalité,Voile,Skipper,Bateau,date
0,1,15:30 FR\r\n,46.407667,-1.841333,241°,17.7 kts,17.5 kts,0.3 nm,357°,0.0 kts,...,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm,FRA,18,louis burton,Bureau Vallée 2,2020-11-08 14:00:00
1,2,15:31 FR\r\n1min,46.405667,-1.830333,241°,11.1 kts,10.9 kts,0.4 nm,357°,0.0 kts,...,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm,MON,10,boris herrmann,Seaexplorer - Yacht Club De Monaco,2020-11-08 14:00:00
2,3,15:30 FR\r\n,46.415167,-1.833167,244°,15.5 kts,15.5 kts,0.5 nm,357°,0.0 kts,...,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm,FRA,8,jeremie beyou,Charal,2020-11-08 14:00:00
3,4,15:30 FR\r\n,46.411833,-1.828000,244°,13.2 kts,13.1 kts,0.7 nm,357°,0.0 kts,...,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm,FRA,59,thomas ruyant,LinkedOut,2020-11-08 14:00:00
4,5,15:30 FR\r\n,46.409833,-1.826000,246°,10.9 kts,10.9 kts,0.2 nm,357°,0.0 kts,...,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm,FRA,53,maxime sorel,V And B Mayenne,2020-11-08 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13666,21,11:30 FR\r\n,-24.688833,-26.735000,348°,14.8 kts,13.3 kts,7.4 nm,348°,14.1 kts,...,12.3 kts,11.4 kts,295.1 nm,4560.7 nm,4431.9 nm,FRA,71,manuel cousin,Groupe Sétin,2021-01-27 11:00:00
13667,22,11:30 FR\r\n,-26.759000,-25.901833,351°,12.7 kts,11.8 kts,6.3 nm,351°,12.0 kts,...,10.2 kts,9.7 kts,245.9 nm,4671.5 nm,4542.7 nm,FRA,50,miranda merron,Campagne de France,2021-01-27 11:00:00
13668,23,11:30 FR\r\n,-26.506333,-29.514167,336°,9.2 kts,7.1 kts,4.6 nm,341°,10.6 kts,...,10.1 kts,8.5 kts,241.9 nm,4704.0 nm,4575.2 nm,FRA,83,clement giraud,Compagnie du lit - Jiliti,2021-01-27 11:00:00
13669,24,11:30 FR\r\n,-48.569167,-54.125667,34°,9.5 kts,9.5 kts,4.7 nm,30°,11.2 kts,...,12.0 kts,11.9 kts,287.7 nm,6368.5 nm,6239.6 nm,FRA,72,alexia barrier,TSE - 4myplanet,2021-01-27 11:00:00


In [16]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13671 entries, 0 to 13670
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Rang           13671 non-null  object        
 1   Heure FR       13671 non-null  object        
 2   Latitude       13671 non-null  float64       
 3   Longitude      13671 non-null  float64       
 4   Cap_30m        13671 non-null  object        
 5   Vitesse_30m    13671 non-null  object        
 6   VMG_30m        13671 non-null  object        
 7   Distance_30m   13671 non-null  object        
 8   Cap_last       13671 non-null  object        
 9   Vitesse_last   13671 non-null  object        
 10  VMG_last       13671 non-null  object        
 11  Distance_last  13671 non-null  object        
 12  Cap_24h        13671 non-null  object        
 13  Vitesse_24h    13671 non-null  object        
 14  VMG_24h        13671 non-null  object        
 15  Distance_24h   1367

In [17]:
url = "https://www.vendeeglobe.org/fr/glossaire"
page = requests.get(url).content
soup = BeautifulSoup(page)

In [18]:
specs_ul = soup.find_all('ul', {"class","boats-list__popup-specs-list"})

In [19]:
dict_list = []
for spec_ul in specs_ul:
    specs_list = [elem.string.split(' : ') for elem in spec_ul.find_all('li')]
    specs_dict = {elem[0]: elem[1] for elem in specs_list}
    skipper = spec_ul.parent()[0].parent()[-5]['href'].split('/')[-1].replace('-', ' ')
    specs_dict["Skipper"] = skipper
    dict_list.append(specs_dict)
specs_df = pd.DataFrame(dict_list).drop(16).reset_index(drop=True)
specs_df.head()

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant,Skipper
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2,fabrice amedeo
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2,romain attanasio
2,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2,alexia barrier
3,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2,yannick bestaven
4,08,NaN,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2,jeremie beyou


In [20]:
func = lambda x: re.findall(r'-?\d+\.?\d*', x)[0] if len(re.findall(r'-?\d+\.?\d*', x)) > 0 else np.nan
numerical_specs = specs_df.iloc[:, [5,6,7,8,10,12,13]].stack().str.replace(',','.').apply(func).unstack()
numerical_specs

,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Hauteur mât,Surface de voiles au près,Surface de voiles au portant
0,18.28,5.85,4.50,7,29,320,570
1,18.28,5.80,4.50,9,28,280,560
2,18.28,5.54,4.50,9,29,260,580
3,18.28,5.80,4.50,8,29,310,550
4,18.28,5.85,4.50,8,29,320,600
5,18.28,5.65,4.50,7.9,29,300,610
6,18.28,5.80,4.50,7.6,28,300,600
7,18.28,5.30,4.50,8.9,26,240,470
8,18.28,5.80,4.50,9,28.50,270,560
9,18.28,5.70,4.5,7.7,29,340,570


NaN values, on utilise un knn pour inferer leur valeur.

In [21]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
numerical_specs = pd.DataFrame(imputer.fit_transform(numerical_specs), columns=numerical_specs.columns)
numerical_specs

,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Hauteur mât,Surface de voiles au près,Surface de voiles au portant
0,18.28,5.85,4.5,7.00,29.0,320.0,570.0
1,18.28,5.80,4.5,9.00,28.0,280.0,560.0
2,18.28,5.54,4.5,9.00,29.0,260.0,580.0
3,18.28,5.80,4.5,8.00,29.0,310.0,550.0
4,18.28,5.85,4.5,8.00,29.0,320.0,600.0
5,18.28,5.65,4.5,7.90,29.0,300.0,610.0
6,18.28,5.80,4.5,7.60,28.0,300.0,600.0
7,18.28,5.30,4.5,8.90,26.0,240.0,470.0
8,18.28,5.80,4.5,9.00,28.5,270.0,560.0
9,18.28,5.70,4.5,7.70,29.0,340.0,570.0


In [22]:
clean_specs_df = specs_df.copy()
clean_specs_df.iloc[:, [5,6,7,8,10,12,13]] = numerical_specs
clean_specs_df['Nombre de dérives'] = clean_specs_df['Nombre de dérives'].str.contains('foil')
clean_specs_df['Date de lancement']
clean_specs_df = clean_specs_df.drop(clean_specs_df.columns[[0,1]], axis = 1)
clean_specs_df['Architecte'].str.lower().replace('[/-]', ' ', regex=True).replace(' +', ' ', regex=True)

0                                      vplp verdier
1                                 bruce farr design
2                                      marc lombard
3                                      verdier vplp
4                                              vplp
5                owen clarke design llp clay oliver
6                                      verdier vplp
7                                owen clarke design
8                           bruce farr yacht design
9                                      verdier vplp
10                                          verdier
11                                     vplp verdier
12                                          lavanos
13                                bruce farr design
14                                     verdier vplp
15                                bruce farr design
16                                   pierre rolland
17                                     verdier vplp
18                               owen clarke design
19          

In [23]:
def clean_architecte(architecte: str):
    if "VPLP" in architecte:
        if architecte:
            pass

# Viz Course

In [107]:
from ipywidgets import widgets
from ipyleaflet import Map, AntPath, WidgetControl

In [69]:
days = [dt.fromordinal(i) for i in range(start_date.toordinal(), (end_date + datetime.timedelta(days=2)).toordinal())]

In [92]:
skipper = race_df['Skipper'].unique()
ordered_race_df = race_df.sort_values(by=['Skipper', 'date'])
coord = ordered_race_df.loc[ordered_race_df['Skipper'] == skipper[0], ['Latitude', 'Longitude']].values.tolist()

In [108]:
m = Map(center=(-10.407666666666664, -1.8413333333333333), zoom=2)

ant_path = AntPath(
    locations=list(coord),
    dash_array=[1, 10],
    delay=1000,
    color='#7590ba',
    pulse_color='#3f6fba'
)

In [109]:

dates = [datetime.date(2015, i, 1) for i in range(1, 13)]
options = [(i.strftime('%b'), i) for i in dates]
tmp = start_date

def on_value_change(change):
    tmp = change['new']
    ant_path.locations = ordered_race_df.loc[(ordered_race_df['Skipper'] == skipper[0]) & (ordered_race_df['date'] <= tmp), ['Latitude', 'Longitude']].values.tolist()

date_slider = widgets.SelectionSlider(
    options=days,
    description='Avancée de la course',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True
)
date_slider.observe(on_value_change, names='value')

In [ ]:
m.add_layer(ant_path)
widget_control1 = WidgetControl(widget=date_slider, position='topright')
m.add_control(widget_control1)
m

In [89]:
print(tmp)

2020-11-22 00:00:00
